In [113]:
import pandas as pd

In [114]:
data = pd.read_csv('test.csv', sep='|')

/home/jmckenzie/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [115]:
#strip whitespace for the column headers
data.columns = [i.strip(' ') for i in data.columns]

In [116]:
to_keep = ['bet_type', 'period', 'quote_time', 'handicap', 'bookie', 'price1', 'price2', 'price3', 'home_team',
          'away_team', 'match_datetime', 'home_full_time_goals', 'away_full_time_goals']

In [117]:
df = data[to_keep]
df = df.dropna(axis=0, subset=['bet_type'])

In [118]:
#strip whitespace from the bet_type column and then remove the asian handicap bets
def remove_whitespace(entry):
    return entry.strip(' ')

df['bet_type'] = df.bet_type.apply(remove_whitespace)
df = df[df.bet_type != 'AH']

In [123]:
def odds_to_probs(odds):
    return 1.0 / odds

In [127]:
df['overound'] = df.price1.apply(odds_to_probs) + df.price2.apply(odds_to_probs)
df['prob1'] = df.price1.apply(odds_to_probs)
df['prob2'] = df.price2.apply(odds_to_probs)

df['implied_1'] = df.prob1 / df.overound
df['implied_2'] = df.prob2 / df.overound

In [131]:
df.head()

,bet_type,period,quote_time,handicap,bookie,price1,price2,price3,home_team,away_team,match_datetime,home_full_time_goals,away_full_time_goals,overound,prob1,prob2,implied_1,implied_2
16,OU,FT,C,0.5,bet365,1.03,15.0,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.037540,0.970874,0.066667,0.935745,0.064255
17,OU,FT,C,1.5,bet365,1.18,4.5,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.069680,0.847458,0.222222,0.792254,0.207746
18,OU,FT,C,2.5,bet365,1.60,2.3,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.059783,0.625000,0.434783,0.589744,0.410256
19,OU,FT,C,3.5,bet365,2.50,1.5,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.066667,0.400000,0.666667,0.375000,0.625000
20,OU,FT,C,4.5,bet365,4.33,1.2,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.064280,0.230947,0.833333,0.216998,0.783002


In [134]:
import math

In [136]:
x = math.log(0.07) * -1
print(x)

2.659260036932778


In [141]:
x = -math.log(1 - 0.97)
print(x)

3.506557897319981


In [142]:
from statsmodels.discrete.discrete_model import Poisson